In [1]:
%matplotlib inline

In [2]:
import sys
sys.path.append("../../")

In [3]:
import numpy as np
import torch
import pandas as pd
from copy import deepcopy
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from uq360.algorithms.homoscedastic_gaussian_process_regression import HomoscedasticGPRegression
from uq360.algorithms.ucc_recalibration import UCCRecalibration
from uq360.metrics import picp, mpiw, compute_regression_metrics

In [4]:
from uq360.datasets import MEPSDataset

In [ ]:
data = MEPSDataset().data()

In [ ]:
categorical_features = ['REGION','SEX','MARRY',
                                 'FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX',
                                 'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                                 'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                                 'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42','ADSMOK42',
                                 'PHQ242','EMPST','POVCAT','INSCOV']

In [ ]:
data = pd.get_dummies(data, columns=categorical_features, prefix_sep='=')

In [ ]:
data = data[data["UTILIZATION"] >=0]

In [ ]:
# Reset index to consecutive integers
data.reset_index(drop=True, inplace=True)
# Drop panel number (not meant to be predictive) and sample weights
data.drop(columns = ['PERWT15F'], inplace=True)
data

In [ ]:
def race(row):
    if row['RACE'] == 'White':
        return 1.0
    return 0.0

In [ ]:
data["RACE"] = data.apply(lambda row: race(row), axis=1)

In [ ]:
data = data.dropna()

In [ ]:
# Separate target variable
y = np.log( 1.0 + data.pop('UTILIZATION'))
a = data['RACE']
# Split data into training and test sets
from sklearn.model_selection import train_test_split
df_train, df_test, y_train, y_test, a_train, a_test = train_test_split(data, y, a, random_state=0)
df_train, df_val, y_train, y_val, a_train, a_val = train_test_split(df_train, y_train, a_train, random_state=0)

In [ ]:
#kwargs = {'cumulative': False}
#sns.distplot(y, hist_kws=kwargs, kde_kws=kwargs)

In [ ]:
normalizer_y = StandardScaler()
y_train = normalizer_y.fit_transform(y_train.values.reshape(-1, 1))
y_test = normalizer_y.transform(y_test.values.reshape(-1, 1))
normalizer_X = MinMaxScaler()
X_train = normalizer_X.fit_transform(df_train)
X_test = normalizer_X.transform(df_test)

In [ ]:
#kwargs = {'cumulative': False}
#sns.distplot(y_test, hist_kws=kwargs, kde_kws=kwargs)

In [ ]:
def create_res_df(method_name, y_test, y_mean, y_lower, y_upper):
    results = {"Overall": compute_regression_metrics(y_test, y_mean, y_lower, y_upper)}
    results["White"] = compute_regression_metrics(y_test[a_test==1.0], y_mean[a_test==1.0], y_lower[a_test==1.0], y_upper[a_test==1.0])
    results["Non-white"] = compute_regression_metrics(y_test[a_test==0.0], y_mean[a_test==0.0], y_lower[a_test==0.0], y_upper[a_test==0.0])

    total = [method_name,
             "Overall",
             results["Overall"]["rmse"],
             results["Overall"]["r2"],
             results["Overall"]["picp"],
             results["Overall"]["mpiw"],
             100*results["Overall"]["auucc_gain"]]
    white = [method_name,
             "White",
             results["White"]["rmse"],
             results["White"]["r2"],
             results["White"]["picp"],
             results["White"]["mpiw"],
             100*results["White"]["auucc_gain"]]
    nonwhite = [method_name,
             "Non-white",
             results["Non-white"]["rmse"],
             results["Non-white"]["r2"],
             results["Non-white"]["picp"],
             results["Non-white"]["mpiw"],
             100*results["Non-white"]["auucc_gain"]]
    res = pd.DataFrame([total, white, nonwhite], columns=['Method', 'Group', 'rmse', 'r2', 'Avg. Coverage', 'Avg. Width', '% AUUCC Gain'])
    return res

In [ ]:
all_results = pd.DataFrame(columns=['Method', 'Group', 'rmse', 'r2', 'Avg. Coverage', 'Avg. Width', '% AUUCC Gain'])

# Train GP Regression

In [ ]:
gp_model = HomoscedasticGPRegression()

In [ ]:
ids = np.random.randint(X_train.shape[0], size=1000)
gp_model = gp_model.fit(X_train[ids], y_train[ids])

In [ ]:
gp_y_test_mean, gp_y_test_lower, gp_y_test_upper = gp_model.predict(X_test)[:3]

In [ ]:
gp_res = create_res_df("GP", y_test.squeeze(), gp_y_test_mean, gp_y_test_lower, gp_y_test_upper)

In [ ]:
gp_res

# Post-hoc calibration using a held out set

In [ ]:
# Let us now train UCCRecalibration models to calibrate BUQ model on white group separately to achieve equalized 
# coverage. We use the the held out calibration to train these models.

In [ ]:
y_val = normalizer_y.transform(y_val.values.reshape(-1, 1))
X_val = normalizer_X.transform(df_val)

In [ ]:
calib_gp_for_white = UCCRecalibration(base_model=gp_model).fit(X_val[a_val==1.0], y_val[a_val==1.0])
calib_gp_y_test_mean_white, calib_gp_y_test_lower_white, calib_gp_y_test_upper_white = calib_gp_for_white.predict(X_test[a_test==1.0], missrate=0.03)[:3]

res_calib_gp_white = compute_regression_metrics(y_test[a_test==1.0].squeeze(), calib_gp_y_test_mean_white, calib_gp_y_test_lower_white, calib_gp_y_test_upper_white)

In [ ]:
gp_res.append(pd.Series(["GP",
             "Calib White",
             res_calib_gp_white["rmse"],
             res_calib_gp_white["r2"],
             res_calib_gp_white["picp"],
             res_calib_gp_white["mpiw"],
             100*res_calib_gp_white["auucc_gain"]], index = gp_res.columns), ignore_index=True)